## <b>RAG with Azure OpenAI<b>

### Prerequisite package installation 

In [ ]:
# ! pip install llama-index-embeddings-azure-openai
# ! pip install llama-index-llms-azure-openai

In [ ]:
# ! pip install llama-index

###  Loading Dotenv package to load environment variables from .env file

In [154]:
from dotenv import load_dotenv
load_dotenv()

True

### Loading Necessary packages

In [180]:
import os
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,StorageContext,load_index_from_storage,get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Here, we setup the embedding model (for retrieval) and llm (for text generation).
Note that you need not only model names (e.g. "text-embedding-ada-002"), but also model deployment names (the one you chose when deploying the model in Azure.
You must pass the deployment name as a parameter when you initialize `AzureOpenAI` and `OpenAIEmbedding`.

### Initializing the AzureOpenAI and OpenAIEmbedding models 

In [156]:
api_key = os.getenv('AZURE_API_KEY')
azure_endpoint = os.getenv('AZURE_ENDPOINT')
api_version = "2023-07-01-preview"

llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    temperature=0.4,
    max_tokens=200
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version
)

In [157]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [158]:
llm

AzureOpenAI(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x0000022A4F3A8450>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x0000022A66D196C0>, completion_to_prompt=<function default_completion_to_prompt at 0x0000022A66D9E980>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='gpt-35-turbo', temperature=0.4, max_tokens=200, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='d63a5e9efcbf4026975ce9e3e0b4a7af', api_base='https://api.openai.com/v1', api_version='2023-07-01-preview', engine='gpt-35-turbo', azure_endpoint='https://mathurai.openai.azure.com/', azure_deployment=None, use_azure_ad=False)

In [159]:
embed_model

AzureOpenAIEmbedding(model_name='text-embedding-ada-002', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x0000022A4F3A8450>, additional_kwargs={}, api_key='d63a5e9efcbf4026975ce9e3e0b4a7af', api_base='https://api.openai.com/v1', api_version='2023-07-01-preview', max_retries=10, timeout=60.0, default_headers=None, reuse_client=True, dimensions=None, azure_endpoint='https://mathurai.openai.azure.com/', azure_deployment='text-embedding-ada-002')

### Data Ingestion from source directory

In [160]:
documents = SimpleDirectoryReader("./data").load_data()

In [161]:
len(documents)

42

### Splitting documents into chunks

In [162]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=100,chunk_overlap=0)

nodes = parser.get_nodes_from_documents(documents)

In [163]:
len(nodes)

230

### Importing FAISS Vector Store

In [164]:
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

In [175]:
import faiss

# dimensions of text-ada-embedding-002
d = 1536

# Creating an index with flat indices and metric as 'Cosine similarity' (IP: Inner Product)
faiss_index= faiss.IndexFlatIP(d)

### Indexing the documents and creating the vector store

In [172]:
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context, 
    embed_model=embed_model, 
    show_progress=True,
)

Generating embeddings:   0%|          | 0/230 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings:   4%|▍         | 10/230 [00:01<00:42,  5.21it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings:   9%|▊         | 20/230 [00:02<00:23,  9.11it/s]

...

Generating embeddings:  91%|█████████▏| 210/230 [00:13<00:01, 19.15it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings:  96%|█████████▌| 220/230 [00:13<00:00, 19.18it/s]

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Generating embeddings: 100%|██████████| 230/230 [00:14<00:00, 16.20it/s]


In [134]:
print(storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x0000022A6B604350>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x0000022A6B604C90>, vector_stores={'default': FaissVectorStore(stores_text=False, is_embedding_query=True), 'image': <llama_index.core.vector_stores.simple.SimpleVectorStore object at 0x0000022A6B604650>}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x0000022A6B605490>)


### Storing the index on the disk for future use

In [173]:
index.storage_context.persist()

### Loading the index from the disk

In [174]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context)

INFO:root:Loading llama_index.vector_stores.faiss.base from ./storage\default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage\default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage\default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage\default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage\default__vector_store.json.
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [176]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)
response_synthesizer = get_response_synthesizer()

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

In [98]:
from llama_index.core.evaluation import FaithfulnessEvaluator
evaluator = FaithfulnessEvaluator(llm=llm)

In [182]:
response = query_engine.query("who is the president of india")

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://mathurai.openai

In [183]:
display(Markdown(f"<b>{response}</b>"))

<b>I'm sorry, I cannot answer that question as it is not related to the provided context information about life insurance and personal information.</b>

In [168]:
response = query_engine.query("what was the previous answer")

INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://mathurai.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://mathurai.openai.azu

In [169]:
display(Markdown(f"<b>{response}</b>"))

<b>There is no previous answer provided in the given context information.</b>